In [9]:
import csv
import os
import numpy as np
import pandas as pd
import sklearn.metrics as mm
import sys
# sir_inference imports
from sir_model import FastProximityModel, patient_zeros_states
from ranking import csr_to_list
import os.path
from os import path
N=50000
## new try with 100 spreaders
N_patient_zero = 20;
lamb = 0.03;
mu = 0.05;
scale=1.0; # Easy Case
T=100;
seed=2
location="networks"
# SIR parameters
t1 = 10;
#seed=int();

In [11]:
## import sys
from time import time
import numpy as np
import pandas as pd
import os.path
from os import path
from scenario import Scenario
from sir_model import EpidemicModel, patient_zeros_states, symptomatic_individuals
from ranking import RANKINGS
log_fname="interactions_proximity_N%dK_s%.1f_T%d_lamb%.2f_s%d.csv"%(N/1000,scale,T,lamb,seed)
csv_file=location+"/"+log_fname
print("seed = %d"%seed, flush=True)
np.random.seed(seed);
if path.exists(csv_file): print("Loading model from %s"%csv_file)
else:
    print("Could not find the file! Was looking for \n "+csv_file+"\n Bye-Bye")
    sys.exit()
print("Load Proximity model", flush=True)
tic = time()
model = EpidemicModel(initial_states=np.zeros(N), x_pos=np.zeros(N), y_pos=np.zeros(N))
model.load_transmissions(csv_file)
model.recover_probas = mu*np.ones(N)
print(f"Loading took {time()-tic:.1f}s",flush=True)
model.initial_states = patient_zeros_states(N, N_patient_zero)
model.time_evolution(model.recover_probas, model.transmissions, print_every=50)
t_max = len(model.transmissions)
print("Save plain dynamics", flush=True)
db = pd.DataFrame()
db["S"] = np.sum(model.states==0,axis=1)
db["I"] = np.sum(model.states==1,axis=1)
db["R"] = np.sum(model.states==2,axis=1)
db.to_csv("csv/Proximity_N%dK_T%d_s1_pz%d_mu%.2f_l%.2f_seed%d.csv"%(N/1000,T,N_patient_zero,mu,lamb,seed),
          index=False, sep="\t")
del db
model.initial_states = model.states[t1]
model.states = model.states[t1:]
model.transmissions = model.transmissions[t1:]

seed = 2
Loading model from networks/interactions_proximity_N50K_s1.0_T100_lamb0.03_s2.csv
Load Proximity model
Loading transmissions from networks/interactions_proximity_N50K_s1.0_T100_lamb0.03_s2.csv
Loading took 30.1s
Running SIR simulation
t = 0 / 100
t = 50 / 100
Save plain dynamics


In [17]:
# trac parameters
trac_tau = 5;
# MF parameters
MF_tau = 5;
MF_delta = 15;
# observation parameters
n_ranking = 50
p_untracked=0
#seed = int(sys.argv[2]);
#seeds for running [32,123,456]
#seed=int(sys.argv[1]);
################################################

intervention_options=dict(quarantine_time=T-t1)
observation_options=dict(n_random=0,n_infected=0,n_ranking=n_ranking, p_symptomatic=0.5, tau=5, p_untracked=p_untracked)

In [18]:
import imp
import scenario
imp.reload(scenario)
from scenario import Scenario
##### RANDOM SCENARIO #######
scenario_rnd = Scenario(
    model, seed=seed+1, 
    ranking_options=dict(ranking=RANKINGS["random"]),
    observation_options=observation_options,
    intervention_options=intervention_options
)
scenario_rnd.run(t_max-t1,  print_every = 1)
print("Save random strategy", flush=True)
scenario_rnd.counts.to_csv("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_rnd.csv"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed,n_ranking),
          index=False, sep="\t")
del scenario_rnd

Using ranking_random to rank
Using seed=3
Selecting 0/50000 untracked individuals
t = 0 / 90
state: S:49924, I:60, R:16 I+R:76
obser: S:50, I:0, R:0, I+R:0
free birds: 60
t = 1 / 90
state: S:49916, I:66, R:18 I+R:84
obser: S:100, I:0, R:0, I+R:0
free birds: 66
t = 2 / 90
state: S:49911, I:69, R:20 I+R:89
obser: S:150, I:0, R:0, I+R:0
free birds: 69
t = 3 / 90
state: S:49904, I:73, R:23 I+R:96
obser: S:200, I:0, R:0, I+R:0
free birds: 73
t = 4 / 90
state: S:49890, I:84, R:26 I+R:110
obser: S:250, I:0, R:0, I+R:0
free birds: 84
t = 5 / 90
state: S:49875, I:97, R:28 I+R:125
obser: S:300, I:0, R:0, I+R:0
free birds: 97
t = 6 / 90
state: S:49861, I:104, R:35 I+R:139
obser: S:350, I:3, R:1, I+R:4
free birds: 101
t = 7 / 90
state: S:49842, I:118, R:40 I+R:158
obser: S:400, I:5, R:1, I+R:6
free birds: 113
t = 8 / 90
state: S:49821, I:134, R:45 I+R:179
obser: S:450, I:8, R:1, I+R:9
free birds: 126
t = 9 / 90
state: S:49804, I:144, R:52 I+R:196
obser: S:500, I:12, R:4, I+R:16
free birds: 132
t =

In [19]:
##### TRACING SCENARIO #######
scenario_trac = Scenario( model, seed=seed+1, 
    ranking_options=dict(ranking=RANKINGS["tracing"], tau=5),
    observation_options=observation_options,
    intervention_options=intervention_options
)
scenario_trac.run(t_max-t1, print_every = 1)
print("Save tracing strategy", flush=True)
scenario_trac.counts.to_csv("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_trac_t%d.csv"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed,n_ranking,trac_tau),
                  index=False, sep="\t")
del scenario_trac

Using ranking_tracing to rank
Using seed=3
Selecting 0/50000 untracked individuals
t = 0 / 90
state: S:49924, I:60, R:16 I+R:76
obser: S:50, I:0, R:0, I+R:0
free birds: 60
t = 1 / 90
state: S:49916, I:66, R:18 I+R:84
obser: S:100, I:0, R:0, I+R:0
free birds: 66
t = 2 / 90
state: S:49911, I:69, R:20 I+R:89
obser: S:150, I:0, R:0, I+R:0
free birds: 69
t = 3 / 90
state: S:49904, I:73, R:23 I+R:96
obser: S:200, I:0, R:0, I+R:0
free birds: 73
t = 4 / 90
state: S:49890, I:84, R:26 I+R:110
obser: S:250, I:0, R:0, I+R:0
free birds: 84
t = 5 / 90
state: S:49875, I:97, R:28 I+R:125
obser: S:300, I:0, R:0, I+R:0
free birds: 97
t = 6 / 90
state: S:49862, I:104, R:34 I+R:138
obser: S:350, I:4, R:0, I+R:4
free birds: 100
t = 7 / 90
state: S:49853, I:109, R:38 I+R:147
obser: S:389, I:11, R:6, I+R:17
free birds: 98
t = 8 / 90
state: S:49829, I:127, R:44 I+R:171
obser: S:432, I:17, R:10, I+R:27
free birds: 110
t = 9 / 90
state: S:49810, I:141, R:49 I+R:190
obser: S:474, I:29, R:13, I+R:42
free birds: 1

In [20]:
##### MF SCENARIO #######
import imp
import scenario
imp.reload(scenario)
from scenario import Scenario
scenario_MF = Scenario(
    model, seed=seed+1, 
    ranking_options=dict(ranking=RANKINGS["backtrack"], 
                         algo="MF", init="all_S", tau=5, delta=10), 
    observation_options=observation_options,
    intervention_options=intervention_options,
)
scenario_MF.run(t_max-t1, print_every = 1)
print("Save MF strategy", flush=True)
scenario_MF.counts.to_csv("csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_MF_t%d_d%d.csv"%(N/1000,T,t1,N_patient_zero,mu,lamb,seed,n_ranking,MF_tau,MF_delta),
                  index=False, sep="\t")
del scenario_MF
# 1h01min per round
print("End seed", flush=True)

Using ranking_backtrack to rank
Using seed=3
Selecting 0/50000 untracked individuals
t = 0 / 90
state: S:49924, I:60, R:16 I+R:76
obser: S:50, I:0, R:0, I+R:0
free birds: 60
t = 1 / 90
state: S:49916, I:66, R:18 I+R:84
obser: S:100, I:0, R:0, I+R:0
free birds: 66
t = 2 / 90
state: S:49911, I:69, R:20 I+R:89
obser: S:150, I:0, R:0, I+R:0
free birds: 69
t = 3 / 90
state: S:49904, I:73, R:23 I+R:96
obser: S:200, I:0, R:0, I+R:0
free birds: 73
t = 4 / 90
state: S:49890, I:84, R:26 I+R:110
obser: S:250, I:0, R:0, I+R:0
free birds: 84
t = 5 / 90
state: S:49875, I:97, R:28 I+R:125
obser: S:300, I:0, R:0, I+R:0
free birds: 97
t = 6 / 90
state: S:49861, I:104, R:35 I+R:139
obser: S:350, I:3, R:1, I+R:4
free birds: 101
t = 7 / 90
state: S:49842, I:118, R:40 I+R:158
obser: S:400, I:5, R:1, I+R:6
free birds: 113
t = 8 / 90
state: S:49821, I:134, R:45 I+R:179
obser: S:450, I:8, R:1, I+R:9
free birds: 126
t = 9 / 90
state: S:49804, I:144, R:52 I+R:196
obser: S:500, I:12, R:4, I+R:16
free birds: 132


In [21]:
import sys
sys.path.insert(0,'../loop_abm/src/')
import abm_utils
import sib
import scipy
#imp.reload(winbp_prob0_rank)
from loop_ranker import sib_rank, sib_drop_rank, greedy_rank, dotd_rank, mean_field_rank, tracing_rank, winbp_rank, winbp_prob0_rank

class dummy_logger():
    def info(self, x):
        print(x)
        
class bp_ranker_class(winbp_prob0_rank.WinBPProb0Ranker):
    
    def step_scenario(self, t, model, obs, params):
        data = {}
        data["logger"] = dummy_logger()
        daily_contacts = []
        if t>0:
            daily_contacts = [(i, j, t-1, l) for i, j, l in list(csr_to_list(model.transmissions[t-1]))]
        daily_obs = []
        print("SCENARIO BP")
        
        if len(obs)!=0:
            obs = pd.DataFrame(obs)
            #print(observations)
            obs = obs[obs["t_test"] == t-1]
            #print(t, observations)
            daily_obs = list(obs[["i", "s", "t_test"]].itertuples(index=False, name=None))  
            print(t, "obs", len(obs))
        #print(t, daily_contacts[0])
        #print(t, daily_obs)
        rank_algo = self.rank(t, daily_contacts, daily_obs, data)
        rank = sorted(rank_algo, key= lambda tup: tup[1], reverse=True)
        rank = [(int(tup[0]), tup[1])  for tup in rank]
        rank_pd = pd.DataFrame(rank, columns=["i", "score"])
        rank_pd["rank"] = range(model.N)
        #print(rank_pd)
        return rank_pd

In [22]:
import sys
sys.path.insert(0,'../loop_abm/src/')
import abm_utils
import sib
import scipy
#imp.reload(winbp_prob0_rank)
from loop_ranker import sib_rank, sib_drop_rank, greedy_rank, dotd_rank, mean_field_rank, tracing_rank, winbp_rank, winbp_prob0_rank

class dummy_logger():
    def info(self, x):
        print(x)
        
class bp_ranker_class_plus(winbp_prob0_rank.WinBPProb0Ranker):
    
    def step_scenario(self, t, model, obs, params):
        data = {}
        data["logger"] = dummy_logger()
        daily_contacts = []
        if t>0:
            daily_contacts = [(i, j, t-1, l) for i, j, l in list(csr_to_list(model.transmissions[t]))]
        daily_obs = []
        print("SCENARIO BP")
        
        if len(obs)!=0:
            obs = pd.DataFrame(obs)
            #print(observations)
            obs = obs[obs["t_test"] == t-1]
            #print(t, observations)
            daily_obs = list(obs[["i", "s", "t_test"]].itertuples(index=False, name=None))  
            print(t, "obs", len(obs))
        #print(t, daily_contacts[0])
        #print(t, daily_obs)
        rank_algo = self.rank(t, daily_contacts, daily_obs, data)
        rank = sorted(rank_algo, key= lambda tup: tup[1], reverse=True)
        rank = [(int(tup[0]), tup[1])  for tup in rank]
        rank_pd = pd.DataFrame(rank, columns=["i", "score"])
        rank_pd["rank"] = range(model.N)
        #print(rank_pd)
        return rank_pd

In [23]:
import sys
sys.path.insert(0,'../loop_abm/src/')
import abm_utils
import sib
import scipy
#imp.reload(winbp_prob0_rank)
from loop_ranker import sib_rank, sib_drop_rank, greedy_rank, dotd_rank, mean_field_rank, tracing_rank, winbp_rank, winbp_prob0_rank

class dummy_logger():
    def info(self, x):
        print(x)
        
class bp_ranker_class_ok(winbp_prob0_rank.WinBPProb0Ranker):
    
    def step_scenario(self, t, model, obs, params):
        data = {}
        data["logger"] = dummy_logger()
        daily_contacts = []
        daily_contacts = [(i, j, t, l) for i, j, l in list(csr_to_list(model.transmissions[-1]))]
        daily_obs = []
        print("SCENARIO BP")
        
        if len(obs)!=0:
            obs = pd.DataFrame(obs)
            #print(observations)
            obs = obs[obs["t_test"] == t-1]
            #print(t, observations)
            daily_obs = list(obs[["i", "s", "t_test"]].itertuples(index=False, name=None))  
            print(t, "obs", len(obs))
        #print(t, daily_contacts[0])
        #print(t, daily_obs)
        rank_algo = self.rank(t, daily_contacts, daily_obs, data)
        rank = sorted(rank_algo, key= lambda tup: tup[1], reverse=True)
        rank = [(int(tup[0]), tup[1])  for tup in rank]
        rank_pd = pd.DataFrame(rank, columns=["i", "score"])
        rank_pd["rank"] = range(model.N)
        #print(rank_pd)
        return rank_pd

In [10]:
mu_r = np.log(mu+1)
mu_r

0.04879016416943205

In [ ]:
import imp
import scenario
imp.reload(scenario)
from scenario import Scenario
name_csv = "csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_bp%d.csv"%(N, T,t1,N_patient_zero,mu,lamb,seed,n_ranking,trac_tau)
pseed=50/N
psus = 0.55
pautoinf = 1e-10
bp_ranker = bp_ranker_class(params = sib.Params(
                                 prob_r = sib.Exponential(mu=mu_r),
                                 pseed = pseed,
                                 psus = psus,
                                 pautoinf = pautoinf),
                 maxit0 = 20,
                 maxit1 = 20,
                 tol = 1e-3,
                 memory_decay = 1e-5,
                 window_length = 21,
                           )
bp_ranker.init(N, T)
bp_ranker.__name__ = "bp"
scenario_bp = Scenario( model, seed=seed, 
    ranking_options=dict(ranking=bp_ranker.step_scenario),
    observation_options=observation_options,
    intervention_options=intervention_options,
                         save_csv = name_csv
)
scenario_bp.run(t_max-t1, print_every = 1)
print("Save bp strategy", flush=True)
scenario_bp.counts.to_csv(name_csv,
          index=False, sep="\t")


Using step_scenario to rank
Using seed=2
Selecting 0/50000 untracked individuals
SCENARIO BP
winBP: (S,I,R): (49909.3, 86.4, 4.3), seeds: 90.7, ll: -29801.0
t = 0 / 90
state: S:49924, I:60, R:16 I+R:76
obser: S:49, I:0, R:1, I+R:1
free birds: 60
SCENARIO BP
1 obs 50
winBP: (S,I,R): (49898.9, 91.2, 9.9), seeds: 88.9, ll: -29813.3
t = 1 / 90
state: S:49914, I:67, R:19 I+R:86
obser: S:99, I:0, R:1, I+R:1
free birds: 67
SCENARIO BP
2 obs 50
winBP: (S,I,R): (49889.9, 95.5, 14.6), seeds: 85.9, ll: -29816.4
t = 2 / 90
state: S:49905, I:72, R:23 I+R:95
obser: S:149, I:0, R:1, I+R:1
free birds: 72
SCENARIO BP
3 obs 50
winBP: (S,I,R): (49881.4, 99.2, 19.4), seeds: 82.9, ll: -29829.8
t = 3 / 90
state: S:49893, I:83, R:24 I+R:107
obser: S:199, I:0, R:1, I+R:1
free birds: 83
SCENARIO BP
4 obs 50
winBP: (S,I,R): (49874.8, 101.0, 24.1), seeds: 79.6, ll: -29823.4
t = 4 / 90
state: S:49885, I:85, R:30 I+R:115
obser: S:249, I:0, R:1, I+R:1
free birds: 85
SCENARIO BP
5 obs 50
winBP: (S,I,R): (49868.6, 10

In [26]:
model.transmissions[0]

<50000x50000 sparse matrix of type '<class 'numpy.float64'>'
	with 298094 stored elements in Compressed Sparse Row format>

In [11]:
import imp
import scenario
imp.reload(scenario)
from scenario import Scenario
name_csv = "csv/Proximity_N%dK_T%d_s1_ti%d_pz%d_mu%.2f_l%.2f_seed%d_obs%d_bp_ok%d.csv"%(N, T,t1,N_patient_zero,mu,lamb,seed,n_ranking,trac_tau)
pseed=50/N
psus = 0.65
pautoinf = 1e-10
bp_ranker = bp_ranker_class_ok(params = sib.Params(
                                 prob_r = sib.Exponential(mu=mu_r),
                                 pseed = pseed,
                                 psus = psus,
                                 pautoinf = pautoinf),
                 maxit0 = 20,
                 maxit1 = 20,
                 tol = 1e-3,
                 memory_decay = 1e-5,
                 window_length = 21,
                           )
bp_ranker.init(N, T)
bp_ranker.__name__ = "bp"
scenario_bp = Scenario( model, seed=seed, 
    ranking_options=dict(ranking=bp_ranker.step_scenario),
    observation_options=observation_options,
    intervention_options=intervention_options,
                         save_csv = name_csv
)
scenario_bp.run(t_max-t1, print_every = 1)
print("Save bp strategy", flush=True)
scenario_bp.counts.to_csv(name_csv,
          index=False, sep="\t")


Using step_scenario to rank
Using seed=2
Selecting 0/50000 untracked individuals
SCENARIO BP
winBP: (S,I,R): (49929.0, 67.4, 3.7), seeds: 71.0, ll: -21468.1
t = 0 / 90
state: S:49924, I:60, R:16 I+R:76
obser: S:99, I:1, R:0, I+R:1
free birds: 59
SCENARIO BP
1 obs 100
winBP: (S,I,R): (49927.9, 64.9, 7.2), seeds: 66.5, ll: -21480.4
t = 1 / 90
state: S:49914, I:67, R:19 I+R:86
obser: S:199, I:1, R:0, I+R:1
free birds: 66
SCENARIO BP
2 obs 100
winBP: (S,I,R): (49928.9, 60.7, 10.4), seeds: 61.4, ll: -21485.3
t = 2 / 90
state: S:49906, I:71, R:23 I+R:94
obser: S:299, I:1, R:0, I+R:1
free birds: 70
SCENARIO BP
3 obs 100
winBP: (S,I,R): (49930.4, 56.3, 13.3), seeds: 56.8, ll: -21490.3
t = 3 / 90
state: S:49894, I:82, R:24 I+R:106
obser: S:398, I:2, R:0, I+R:2
free birds: 80
SCENARIO BP
4 obs 100
winBP: (S,I,R): (49931.5, 52.6, 15.9), seeds: 53.6, ll: -21500.8
t = 4 / 90
state: S:49886, I:84, R:30 I+R:114
obser: S:498, I:2, R:0, I+R:2
free birds: 82
SCENARIO BP
5 obs 100
winBP: (S,I,R): (49933.

KeyboardInterrupt: 

In [29]:
daily_contacts = [(i, j, 0, l) for i, j, l in list(csr_to_list(model.transmissions[0]))]

In [30]:
daily_contacts

[(0, 12344, 0, 0.03),
 (0, 19099, 0, 0.03),
 (0, 19857, 0, 0.03),
 (0, 30945, 0, 0.03),
 (0, 40567, 0, 0.03),
 (0, 49228, 0, 0.03),
 (1, 4393, 0, 0.03),
 (1, 6894, 0, 0.03),
 (1, 8578, 0, 0.03),
 (1, 13964, 0, 0.03),
 (1, 47128, 0, 0.03),
 (1, 47728, 0, 0.03),
 (2, 1870, 0, 0.03),
 (2, 9011, 0, 0.03),
 (3, 3775, 0, 0.03),
 (3, 9152, 0, 0.03),
 (3, 15087, 0, 0.03),
 (3, 25290, 0, 0.03),
 (3, 36605, 0, 0.03),
 (3, 37546, 0, 0.03),
 (3, 43034, 0, 0.03),
 (3, 43244, 0, 0.03),
 (4, 1514, 0, 0.03),
 (4, 4541, 0, 0.03),
 (4, 7440, 0, 0.03),
 (4, 10975, 0, 0.03),
 (4, 16081, 0, 0.03),
 (4, 22785, 0, 0.03),
 (4, 29813, 0, 0.03),
 (4, 37191, 0, 0.03),
 (5, 12595, 0, 0.03),
 (6, 4978, 0, 0.03),
 (6, 10594, 0, 0.03),
 (6, 16702, 0, 0.03),
 (6, 23582, 0, 0.03),
 (6, 28224, 0, 0.03),
 (6, 35960, 0, 0.03),
 (6, 46520, 0, 0.03),
 (6, 49762, 0, 0.03),
 (7, 21837, 0, 0.03),
 (7, 28219, 0, 0.03),
 (7, 30319, 0, 0.03),
 (7, 31044, 0, 0.03),
 (7, 47114, 0, 0.03),
 (8, 5896, 0, 0.03),
 (8, 14711, 0, 0.03),
